In [1]:
# here we are mounting our S3 bucket with a specific MOUNT_NAME
try:
  import urllib
  ACCESS_KEY ="XX"
  SECRET_KEY ="XX"
  ENCODED_SECRET_KEY = urllib.parse.quote(SECRET_KEY,"")
  AWS_BUCKET_NAME = "sree-databricks-test1"
  MOUNT_NAME = "s3datav3"
  #dbutils.fs.unmount("/mnt/MOUNT_NAME")
  dbutils.fs.mount("s3n://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)
  display(dbutils.fs.ls("/mnt/%s" % MOUNT_NAME))
except Exception as e:
  print("S3 already mounted") 
  

S3 already mounted

In [2]:
MOUNT_NAME = "s3datav3"
#display(dbutils.fs.ls("/mnt/%s/alphav/" % MOUNT_NAME))
display(dbutils.fs.ls("dbfs:/mnt/%s/results/" % MOUNT_NAME))

path,name,size
dbfs:/mnt/s3datav3/results/r2results.csv,r2results.csv,2335
dbfs:/mnt/s3datav3/results/r2results_withfeatures.csv,r2results_withfeatures.csv,2317
dbfs:/mnt/s3datav3/results/test/,test/,0


In [3]:
#"MMM", "AXP", "AAPL", "BA", "CAT", "CVX", "CSC", "KO", "DIS", "DOW", "XOM", "GS", "HD", "IBM", "INT", "JNJ", "JPM", "MCD", "MRK", "MSF", "NKE", "PFE", "PG", "TRV", "UTX", "UNH", "VZ", "V", "WMT", "WBA"

In [4]:
from datetime import datetime

In [5]:
# We moved this part to the ETL part. this code needs to be removed 
def clean_dataframe(df_data1):
  df_data1 = df_data1.drop('_c0')
  df_data1 = df_data1.drop('1. open')
  df_data1 = df_data1.drop('2. high')
  df_data1 = df_data1.drop('3. low')
  df_data1 = df_data1.drop('5. volume')
  df_data1 = df_data1.drop('VWAP')
  df_data1 = df_data1.withColumnRenamed("4. close","close")
  df_data1.dropna()
  return df_data1

In [6]:
# HERE we are diving the data into train and test
from pyspark.sql.functions import unix_timestamp, lit
def get_train_testdata(df_data1):
  train_data = df_data1.filter(df_data1["DailyDate"] < unix_timestamp(lit('2016-06-01 00:00:00')).cast('timestamp'))
  test_data  = df_data1.filter(df_data1["DailyDate"] > unix_timestamp(lit('2016-06-01 00:00:00')).cast('timestamp'))
  print("Number of training records: " + str(train_data.count()))
  print("Number of testing records : " + str(test_data.count()))
  return train_data, test_data

In [7]:
from pyspark.ml.feature import VectorAssembler,RFormula
from pyspark.ml.regression import LinearRegression, GeneralizedLinearRegression, DecisionTreeRegressor
from pyspark.ml.regression import LinearRegressionModel
from pyspark.ml import Pipeline, Model,PipelineModel
from pyspark.ml.evaluation import RegressionEvaluator 
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder 
import sklearn.metrics
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import monotonically_increasing_id, lag
from pyspark.sql.window import Window

In [8]:

# this is a dataframe which holds the r2 value for each ticker and model associated with it
# in this, initialising
result_summary = spark.range(0).drop("id")
r2_columns = ['ticker', 'LR_R2', 'LR_RMSE','XG_R2','XG_RMSE']
result_summary = spark.createDataFrame([('Test',4.0,5.0,7.0,8.0)], r2_columns)
firstrow = spark.createDataFrame([('Test',4.0,5.0,7.0,8.0)], r2_columns)
result_summary = result_summary.union(firstrow )
result_summary.show()

+------+-----+-------+-----+-------+
ticker|LR_R2|LR_RMSE|XG_R2|XG_RMSE|
+------+-----+-------+-----+-------+
 Test| 4.0| 5.0| 7.0| 8.0|
 Test| 4.0| 5.0| 7.0| 8.0|
+------+-----+-------+-----+-------+

In [9]:
# We are removing the columns that causes a dataleakage 
def removecolumns(train_data):
  columns = train_data.columns
  columns.remove('close_lag7D')
  columns.remove('DailyDate')
  columns.remove('close_lag14D')
  columns.remove('close_lag28D')
  columns.remove('close_lag50D')
  columns.remove('close_lag200D')
  return columns

In [10]:
# HERE we are running the model for all the Dow 30 tickers. 

#stockList = ["MMM", "AAPL"] #["MMM", "AXP", "AAPL", "BA", "CAT"] 
             
stockList = ["MMM", "AXP", "AAPL", "BA", "CAT", "CVX", "KO", "DIS", "XOM", "GS", "HD", "IBM", "INT", "JNJ", "JPM", "MCD", "MRK", "MSF", "NKE", "PFE", "PG", "TRV", "UTX", "UNH", "VZ", "V", "WMT", "WBA"]

MOUNT_NAME = "s3datav3"

for stock in stockList:
  #stockList = ["MMM", "AAPL"] #["MMM", "AXP", "AAPL", "BA", "CAT"] 
  df_data1 = spark.read\
      .format("csv")\
      .option('header', 'true')\
      .option('inferSchema', 'true')\
      .load("/mnt/"+MOUNT_NAME +"/alphaandfeatures/" + stock+ ".csv" )
  #df_data1.head(5)
  df_data2 = df_data1.drop('_c0').dropna()

  train_data, test_data = get_train_testdata(df_data2)
  columns = removecolumns(train_data)
  
 
  #Formula : close_lag7D ~ close + SMA + EMA + MACD_Signal + MACD_Hist + MACD + RSI + SlowD + SlowK + ADX + CCI + dateSMA7 + dateSMA14 + dateSMA28 + dateSMA50 + dateSMA200 + date_converted + DateofMon + Month + Year + WeekSeq + SMA50 + SMA200 + SMA7 + SMA14 + SMA28 + Delta
  
  formula = "{} ~ {}".format("close_lag7D", " + ".join(columns))
  print("Formula : {}".format(formula))
  rformula = RFormula(formula = formula)
  lr = LinearRegression()
  pipeline = Pipeline(stages=[rformula, lr])
  # Parameter grid
  paramGrid = ParamGridBuilder()\
           .addGrid(lr.regParam,[0.01, .04])\
            .build()
  cv = CrossValidator()\
        .setEstimator(pipeline)\
        .setEvaluator(RegressionEvaluator()\
                         .setMetricName("r2"))\
        .setEstimatorParamMaps(paramGrid)\
        .setNumFolds(3)

  cvModel = cv.fit(train_data)
  cvModel.avgMetrics
  predictions = cvModel.transform(test_data)
  evaluator = RegressionEvaluator(labelCol="label",
                                    predictionCol="prediction",
                                    metricName="rmse")

  y_true = predictions.select('label').toPandas()
  y_pred  = predictions.select('prediction').toPandas()

  r2_score = sklearn.metrics.r2_score(y_true, y_pred)
  print('r2_score: {0}'.format(r2_score))

  rmse = evaluator.evaluate(predictions)
  print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

  print(cvModel.explainParams())
  firstrow = spark.createDataFrame([(stock,format(r2_score),format(rmse),format(r2_score),format(rmse))], r2_columns)
  result_summary = result_summary.union(firstrow )

#df_data.show(5)
#df2 = df_data1.toPandas()
#df2.isnull().sum()
#df2 = df_data1.toPandas()
#df2.size
#df2.dtypes
#df2.count()



Number of training records: 1402
Number of testing records : 214
Formula : close_lag7D ~ close + SMA + EMA + MACD_Signal + MACD_Hist + MACD + RSI + SlowD + SlowK + ADX + CCI + dateSMA7 + dateSMA14 + dateSMA28 + dateSMA50 + dateSMA200 + date_converted + DateofMon + Month + Year + WeekSeq + SMA50 + SMA200 + SMA7 + SMA14 + SMA28 + Delta
/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)
r2_score: 0.778889708870703
Root Mean Squared Error (RMSE) on test data = 3.21909
estimator: estimator to be cross-validated (current: Pipeline_3b7239a3fbb8)
estimatorParamMaps: estimator param maps (current: [{Param(parent='LinearRegression_d7fa2b319ea3', name='regParam', doc='regularization parameter (>= 0).'): 0.01}, {Param(parent='LinearRegression_d7fa2b319ea3', name='regParam', doc='regularization parameter (>= 0).'): 0.04}])
evaluator: evaluator used to select hyper-parameters that maximize the validator metric (current: RegressionEvaluator_24e8cab2ef22)
seed: random seed. (default: 7809051150349531440)
Number of training records: 1402
Number of testing records : 214
Formula : close_lag7D ~ close + SMA + EMA + MACD_Signal + MACD_Hist + MACD + RSI + SlowK + SlowD + ADX + CCI + dateSMA7 + dateSMA14 + dateSMA28 + dateSMA50 + dateSMA200 + date_converted + DateofMon + Month + Year + WeekSeq + SMA50 + SMA200 + SMA7 + SMA14 + SMA28 + Delta
/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)
r2_score: 0.8961289527014993
Root Mean Squared Error (RMSE) on test data = 2.18302
estimator: estimator to be cross-validated (current: Pipeline_417d10aaf13b)
estimatorParamMaps: estimator param maps (current: [{Param(parent='LinearRegression_a96b5eb7b5ea', name='regParam', doc='regularization parameter (>= 0).'): 0.01}, {Param(parent='LinearRegression_a96b5eb7b5ea', name='regParam', doc='regularization parameter (>= 0).'): 0.04}])
evaluator: evaluator used to select hyper-parameters that maximize the validator metric (current: RegressionEvaluator_b462878939c2)
seed: random seed. (default: 7809051150349531440)
Number of training records: 1402
Number of testing records : 214
Formula : close_lag7D ~ close + SMA + EMA + MACD + MACD_Hist + MACD_Signal + RSI + SlowK + SlowD + ADX + CCI + dateSMA7 + dateSMA14 + dateSMA28 + dateSMA50 + dateSMA200 + date_converted + DateofMon + Month + Year + WeekSeq + SMA50 + SMA200 + SMA7 + SMA14 + SMA28 + Delta
/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)
r2_score: -0.77004963948451
Root Mean Squared Error (RMSE) on test data = 19.0569
estimator: estimator to be cross-validated (current: Pipeline_55bbc69b3f8f)
estimatorParamMaps: estimator param maps (current: [{Param(parent='LinearRegression_f9ee8232bfd7', name='regParam', doc='regularization parameter (>= 0).'): 0.01}, {Param(parent='LinearRegression_f9ee8232bfd7', name='regParam', doc='regularization parameter (>= 0).'): 0.04}])
evaluator: evaluator used to select hyper-parameters that maximize the validator metric (current: RegressionEvaluator_9913b2a5994c)
seed: random seed. (default: 7809051150349531440)
Number of training records: 1402
Number of testing records : 214
Formula : close_lag7D ~ close + SMA + EMA + MACD_Signal + MACD_Hist + MACD + RSI + SlowD + SlowK + ADX + CCI + dateSMA7 + dateSMA14 + dateSMA28 + dateSMA50 + dateSMA200 + date_converted + DateofMon + Month + Year + WeekSeq + SMA50 + SMA200 + SMA7 + SMA14 + SMA28 + Delta
/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)


In [11]:
pdata = df_data2.toPandas()
pdata.tail(5)

,close,DailyDate,SMA,EMA,MACD_Signal,MACD_Hist,MACD,RSI,SlowK,SlowD,ADX,CCI,close_lag7D,close_lag14D,close_lag28D,close_lag50D,close_lag200D,dateSMA7,dateSMA14,dateSMA28,dateSMA50,dateSMA200,date_converted,DateofMon,Month,Year,WeekSeq,SMA50,SMA200,SMA7,SMA14,SMA28,Delta
1612,83.05,2017-03-31,84.5305,84.1665,-0.2333,-0.2192,-0.4525,41.9830,56.4248,65.8836,18.2914,-78.3132,82.68,84.03,85.50,75.78,75.78,83.400000,83.650909,84.595238,84.988333,83.052929,1.490918e+09,31,3,7.0,348,83.9632,82.67850,83.340000,84.073571,85.160714,1.28470
1613,82.95,2017-04-03,84.3840,84.0506,-0.2843,-0.2040,-0.4883,41.4489,32.7816,53.7774,18.6798,-75.4789,82.57,84.28,85.25,76.00,76.00,83.396667,83.381818,84.455238,85.052571,83.069275,1.491178e+09,3,4,7.0,348,83.9878,82.67750,83.335714,83.898571,85.041429,1.31030
1614,82.50,2017-04-04,84.2095,83.9029,-0.3368,-0.2099,-0.5467,39.0933,17.9678,35.7247,19.2433,-87.6173,82.48,85.92,84.96,76.47,76.47,83.220000,83.188182,84.294286,84.981667,83.077246,1.491264e+09,4,4,7.0,348,84.0068,82.67675,83.268571,83.662857,84.881071,1.33005
1615,81.17,2017-04-05,84.0100,83.6426,-0.4079,-0.2845,-0.6924,33.2197,14.9022,21.8838,20.5496,-151.8662,83.13,85.92,84.51,77.40,77.40,82.791667,83.002727,84.064762,84.918889,83.074928,1.491350e+09,5,4,7.0,348,84.0056,82.66795,82.927143,83.341429,84.651071,1.33765
1616,81.66,2017-04-06,83.8275,83.4538,-0.4782,-0.2813,-0.7595,36.8965,23.9270,18.9323,21.7906,-140.4961,82.82,85.89,81.73,77.02,77.02,82.440000,82.878182,83.898095,84.859722,83.064748,1.491437e+09,6,4,7.0,348,83.9982,82.66025,82.630000,83.037857,84.452500,1.33795


In [12]:
formula = "{} ~ {}".format("close_lag7D", " + ".join(columns))
print("Formula : {}".format(formula))

Formula : close_lag7D ~ close + SMA + EMA + MACD_Signal + MACD_Hist + MACD + RSI + SlowK + SlowD + ADX + CCI + dateSMA7 + dateSMA14 + dateSMA28 + dateSMA50 + dateSMA200 + date_converted + DateofMon + Month + Year + WeekSeq + SMA50 + SMA200 + SMA7 + SMA14 + SMA28 + Delta

In [14]:
#pdata = train_data.toPandas()
#pdata.head(5)

In [15]:
# HERE THE collected data is written to a csv file. 
# This csv file is uploaded to S3

pandas_result_summary = result_summary.toPandas()
pandas_result_summary = pandas_result_summary.iloc[2:]
pandas_result_summary.to_csv("GBr2results_withfeatures.csv")
dbutils.fs.cp('file:/databricks/driver/r2results_withfeatures.csv','dbfs:/mnt/s3datav3/results/r2results_withfeatures.csv')
pandas_result_summary.head(5)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-1344289081075472> in <module> 
 5 pandas_result_summary = pandas_result_summary . iloc [ 2 : ] 
 6 pandas_result_summary . to_csv ( "GBr2results_withfeatures.csv" ) 
 ----> 7 dbutils . fs . cp ( 'file:/databricks/driver/r2results_withfeatures.csv' , 'dbfs:/mnt/s3datav3/results/r2results_withfeatures.csv' ) 
 8 pandas_result_summary . head ( 5 ) 

 /local_disk0/tmp/1588439074297-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 312 exc . __context__ = None 
 313 exc . __cause__ = None 
 --> 314 raise exc
 315 return f_with_exception_handling
 316 

 ExecutionError : An error occurred while calling z:com.databricks.backend.daemon.dbutils.FSUtils.cp.
: java.io.FileNotFoundException: File file:/databricks/driver/r2results_withfeatures.csv does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:611)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:824)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:421)
	at com.databricks.backend.daemon.dbutils.FSUtils$$anonfun$cp$1$$anonfun$apply$3.apply(DBUtilsCore.scala:114)
	at com.databricks.backend.daemon.dbutils.FSUtils$$anonfun$cp$1$$anonfun$apply$3.apply(DBUtilsCore.scala:113)
	at com.databricks.backend.daemon.dbutils.FSUtils$.com$databricks$backend$daemon$dbutils$FSUtils$$withFsSafetyCheck(DBUtilsCore.scala:81)
	at com.databricks.backend.daemon.dbutils.FSUtils$$anonfun$cp$1.apply(DBUtilsCore.scala:113)
	at com.databricks.backend.daemon.dbutils.FSUtils$$anonfun$cp$1.apply(DBUtilsCore.scala:113)
	at com.databricks.backend.daemon.dbutils.FSUtils$.com$databricks$backend$daemon$dbutils$FSUtils$$withFsSafetyCheck(DBUtilsCore.scala:81)
	at com.databricks.backend.daemon.dbutils.FSUtils$.cp(DBUtilsCore.scala:112)
	at com.databricks.backend.daemon.dbutils.FSUtils.cp(DBUtilsCore.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [16]:
#df_data2.show(10)
df_data3 = df_data2['close','DailyDate','close_lag7D']
df_data3.show(10)

+-----+-------------------+-----------+
close| DailyDate|close_lag7D|
+-----+-------------------+-----------+
35.18|2010-11-03 00:00:00| 34.83|
35.95|2010-11-04 00:00:00| 34.51|
35.14|2010-11-05 00:00:00| 34.01|
35.09|2010-11-08 00:00:00| 34.01|
35.24|2010-11-09 00:00:00| 34.42|
35.11|2010-11-10 00:00:00| 34.76|
35.21|2010-11-11 00:00:00| 34.89|
34.83|2010-11-12 00:00:00| 33.98|
34.51|2010-11-15 00:00:00| 34.31|
34.01|2010-11-16 00:00:00| 33.68|
+-----+-------------------+-----------+
only showing top 10 rows

In [17]:
import matplotlib.pyplot as plt  
from pylab import rcParams
import seaborn as sns  

pdata = df_data3.toPandas()
rcParams['figure.figsize'] = 25, 10
fig = plt.figure()

ax1 = sns.lineplot(data=pdata, x="DailyDate", y="close", markers=True, dashes=False,label ='raw close')
#ax1 = sns.lineplot(data=pdata, x="date", y="avg7", markers=True, dashes=False)
ax1 = sns.lineplot(data=pdata, x="DailyDate", y="close_lag7D", markers=True, dashes=False,label ='close_lag7D')
#ax1 = sns.lineplot(data=pdata, x="date", y="avg28", markers=True, dashes=False)
#ax1 = sns.lineplot(data=pdata, x="date", y="avgof14D_lag7D", markers=True, dashes=False,label ='avgof14D_lag7D')
display(fig)

In [19]:
#firstrow = spark.createDataFrame([(stock,format(r2_score),format(rmse),format(r2_score),format(rmse))], r2_columns)
#result_summary = result_summary.union(firstrow )




In [20]:
train_data.show(5)

In [21]:
#from pyspark.sql.functions import monotonically_increasing_id, lag
#from pyspark.sql.window import Window

#write_train_data = train_data['DailyDate','close']
#write_test_data = test_data['DailyDate']
#write_test_data = test_data.withColumn('id', monotonically_increasing_id())
#temp = y_true['label']

#write_test_data = write_test_data.withColumn('y_true', temp)
#write_test_data = write_test_data.withColumn('y_pred', y_pred)




In [22]:
import matplotlib.pyplot as plt  
from pylab import rcParams
import seaborn as sns  

data_zoom = write_train_data #df_stg2[(df_stg2['date'] > '2017-01-01' )]
pdata = data_zoom.toPandas()

rcParams['figure.figsize'] = 25, 10
fig = plt.figure()

ax1 = sns.lineplot(data=write_train_data, x="DailyDate", y="close", markers=True, dashes=False,label ='raw open')
#ax1 = sns.lineplot(data=pdata, x="date", y="avg7", markers=True, dashes=False)
#ax1 = sns.lineplot(data=pdata, x="date", y="avgof14D_lag0D", markers=True, dashes=False,label ='avgof14D_lag0D')
#ax1 = sns.lineplot(data=pdata, x="date", y="avg28", markers=True, dashes=False)
#ax1 = sns.lineplot(data=pdata, x="date", y="avgof14D_lag7D", markers=True, dashes=False,label ='avgof14D_lag7D')
display(fig)

In [23]:
test_data
columns.remove('DailyDate')

In [24]:
print(result_summary)

In [25]:
bestModel = cvModel.bestModel
bestModel.stages[-1]._java_obj.parent().getRegParam()
#bestModel.getParam


In [26]:
bestModel.save("/tmp/rf20200501a")

In [27]:
rfPath = "/tmp/rf20200501a"
sameRFModel = PipelineModel.load(rfPath)

In [28]:
sameRFModel
predictions = sameRFModel.transform(test_data)
evaluator = RegressionEvaluator(labelCol="label",
                                  predictionCol="prediction",
                                  metricName="rmse")

In [29]:
import pandas as pd
import numpy as np
result_summary  = pd.DataFrame([])
for i in np.arange(0, 4):
    if i % 2 == 0:
        result_summary.append(pd.DataFrame({'A': i, 'B': i + 1}, index=[0]), ignore_index=True)
result_summary.head()

In [30]:
print('done')
#%sh
#ls
#%sh
#ls -lrt

In [31]:
cvModel.save('file:/databricks/driver/'+stock+'.csv')

In [32]:
dbutils.fs.mkdirs("/mnt/"+MOUNT_NAME +"/allmodels/")

In [33]:
best_lr = cvModel.bestModel
best_lr.save("/mnt/"+MOUNT_NAME +"/allmodels/testcvmodel/") 

In [34]:
rfPath = "/mnt/"+MOUNT_NAME +"/allmodels/testcvmodel/"
sameRFModel = PipelineModel.load(rfPath)

In [35]:
cvModel.getEstimatorParamMaps()[ np.argmax(cvModel.avgMetrics) ]